In [1]:
import numpy as np 
import os

subid = np.array([])
dx = np.array([])
pred = np.array([])
#features = np.concatenate([np.loadtxt('./features_1.txt'), np.loadtxt('./features_2.txt'), np.loadtxt('./features_3.txt'), np.loadtxt('./features_4.txt'), np.loadtxt('./features_5.txt')])
#print(features.shape)
print(os.getcwd())
for i in range(5):
    subid = np.concatenate([subid, np.genfromtxt('./5cv/subid_' + str(i+1) + '.txt', dtype='str')]) 
    dx = np.concatenate([dx,  np.loadtxt('./5cv/dx_' + str(i+1) + '.txt')]) 
    pred = np.concatenate([pred,  np.loadtxt('./5cv/pred_' + str(i+1) + '.txt')]) 
#    features = np.concatenate([features,  np.loadtxt('./features_' + str(i+1) + '.txt')], axis=0) 


C:\Users\E31195\Dropbox\Stanford\Codes\ABCD\SexDiff


In [2]:
predclass = pred>=0.5
print(sum(predclass==dx)/pred.shape[0])

0.8953831041257367


In [10]:
import pandas as pd
from functools import reduce

demog = pd.read_csv('abcd_ssphp01.csv')
allsub_demog = demog[demog.subjectkey.isin(subid) & demog.eventname.isin(['baseline_year_1_arm_1'])]

volscores = pd.read_csv('volume_scores.csv')
svol = volscores[['subjectkey', 'sri24_suptent_supratentorium_volume']]
preds = pd.DataFrame({'subjectkey': subid, 'pred': pred, 'dx': dx, 'predclass': predclass}, columns=['subjectkey', 'pred', 'dx', 'predclass'])
#dfs = [allsub_demog, preds, svol]
dfs = [allsub_demog, svol]
all_info = reduce(lambda left,right: pd.merge(left,right,on='subjectkey'), dfs)



C:\Users\E31195\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import chisquare

svols = pd.to_numeric(all_info.sri24_suptent_supratentorium_volume)
svolf = svols[all_info['sex'] == 'F']
svolm = svols[all_info['sex'] == 'M']

[sss, p] = stats.ttest_ind(svolf, svolm)
print ('%2.20f' % p)

print (np.mean(svolf/1000), np.std(svolf/1000))
print (np.mean(svolm/1000), np.std(svolm/1000))


0.10422388940708013361
1341.511098844673 15.34797080160998
1342.0783101906322 16.08011252437921


In [11]:
all_info.columns


Index(['collection_id', 'abcd_ssphp01_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_age', 'interview_date', 'sex', 'eventname',
       'sds_p_ss_dims', 'sds_p_ss_dims_nm', 'sds_p_ss_dims_nt', 'sds_p_ss_sbd',
       'sds_p_ss_sbd_nm', 'sds_p_ss_sbd_nt', 'sds_p_ss_da', 'sds_p_ss_da_nm',
       'sds_p_ss_da_nt', 'sds_p_ss_swtd', 'sds_p_ss_swtd_nm',
       'sds_p_ss_swtd_nt', 'sds_p_ss_does', 'sds_p_ss_does_nm',
       'sds_p_ss_does_nt', 'sds_p_ss_shy', 'sds_p_ss_shy_nm',
       'sds_p_ss_shy_nt', 'sds_p_ss_total', 'sds_p_ss_total_nm',
       'sds_p_ss_total_nt', 'pds_p_ss_male_category', 'pds_p_ss_male_cat_nm',
       'pds_p_ss_male_cat_nt', 'pds_p_ss_female_category',
       'pds_p_ss_female_cat_nm', 'pds_p_ss_female_cat_nt', 'cna_p_ss_sum',
       'cna_p_ss_sum_nm', 'cna_p_ss_sum_nt', 'collection_title',
       'sri24_suptent_supratentorium_volume'],
      dtype='object')

In [109]:
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import chisquare

pds_m = (pd.to_numeric(all_info.pds_p_ss_male_category))
pds_f = (pd.to_numeric(all_info.pds_p_ss_female_category))

pf = (pds_f[all_info['sex'] == 'F'])
pm = (pds_m[all_info['sex'] == 'M'])

pfnan = np.isnan(pf)
pmnan = np.isnan(pm)

pf = pf[~pfnan]
pm = pm[~pmnan]

print (np.mean(pf), np.std(pf))
print (np.mean(pm), np.std(pm))

[sss, p] = stats.ttest_ind(pf, pm)

print ('%2.20f' % p)

pdsfreq = np.zeros([2,5])
pdsfreq[0][0] = sum(pf == 1.0)
pdsfreq[0][1] = sum(pf == 2.0)
pdsfreq[0][2] = sum(pf == 3.0)
pdsfreq[0][3] = sum(pf == 4.0)
pdsfreq[0][4] = sum(pf == 5.0)
pdsfreq[1][0] = sum(pm == 1.0)
pdsfreq[1][1] = sum(pm == 2.0)
pdsfreq[1][2] = sum(pm == 3.0)
pdsfreq[1][3] = sum(pm == 4.0)
pdsfreq[1][4] = sum(pm == 5.0)

chi2_stat, p_val, dof, ex = stats.chi2_contingency(pdsfreq)
print ('chi2 = %f; p = %2.20f' % (chi2_stat, p_val))

2.143917307182613 0.9071458376063103
1.3422590068159688 0.5839428476924595
0.00000000000000000000
chi2 = 1819.863862; p = 0.00000000000000000000


In [110]:
# look at incorrectly classified ones 

sf = all_info['sex'] == 'F'
sm = all_info['sex'] == 'M'
cc = all_info['dx'] == all_info['predclass']

pfi = (pds_f[sf & ~cc])
pmi = (pds_m[sm & ~cc])

pfinan = np.isnan(pfi)
pminan = np.isnan(pmi)

pfi = pfi[~pfinan]
pmi = pmi[~pminan]

print (np.mean(pfi), np.std(pfi))
print (np.mean(pmi), np.std(pmi))

[sss, p] = stats.ttest_ind(pfi, pmi)

print ('%2.20f' % p)

pdsfreqi = np.zeros([2,5])
pdsfreqi[0][0] = sum(pfi == 1.0)
pdsfreqi[0][1] = sum(pfi == 2.0)
pdsfreqi[0][2] = sum(pfi == 3.0)
pdsfreqi[0][3] = sum(pfi == 4.0)
pdsfreqi[0][4] = sum(pfi == 5.0)
pdsfreqi[1][0] = sum(pmi == 1.0)
pdsfreqi[1][1] = sum(pmi == 2.0)
pdsfreqi[1][2] = sum(pmi == 3.0)
pdsfreqi[1][3] = sum(pmi == 4.0)
pdsfreqi[1][4] = sum(pmi == 5.0)

chi2_stat, p_val, dof, ex = stats.chi2_contingency(pdsfreqi)
print ('chi2 = %f; p = %2.20f' % (chi2_stat, p_val))

2.046908315565032 0.8989071984964152
1.3654970760233918 0.6336399661610663
0.00000000000000000000
chi2 = 135.788254; p = 0.00000000000000000000


In [111]:
# look at incorrectly classified ones 

sf = all_info['sex'] == 'F'
sm = all_info['sex'] == 'M'
cc = all_info['dx'] == all_info['predclass']

pfc = (pds_f[sf & cc])
pmc = (pds_m[sm & cc])

pfinan = np.isnan(pfc)
pminan = np.isnan(pmc)

pfc = pfc[~pfinan]
pmc = pmc[~pminan]

print (np.mean(pfc), np.std(pfc))
print (np.mean(pmc), np.std(pmc))

[sss, p] = stats.ttest_ind(pfc, pmc)

print ('%2.20f' % p)

pdsfreqc = np.zeros([2,5])
pdsfreqc[0][0] = sum(pfc == 1.0)
pdsfreqc[0][1] = sum(pfc == 2.0)
pdsfreqc[0][2] = sum(pfc == 3.0)
pdsfreqc[0][3] = sum(pfc == 4.0)
pdsfreqc[0][4] = sum(pfc == 5.0)
pdsfreqc[1][0] = sum(pmc == 1.0)
pdsfreqc[1][1] = sum(pmc == 2.0)
pdsfreqc[1][2] = sum(pmc == 3.0)
pdsfreqc[1][3] = sum(pmc == 4.0)
pdsfreqc[1][4] = sum(pmc == 5.0)

chi2_stat, p_val, dof, ex = stats.chi2_contingency(pdsfreqc)
print ('chi2 = %f; p = %2.20f' % (chi2_stat, p_val))

2.157687651331719 0.9074691300815164
1.3401486988847584 0.5791723902733596
0.00000000000000000000
chi2 = 1695.630250; p = 0.00000000000000000000


In [121]:
pdsall = np.zeros([2,10])
pdsall[0][0:5] = pdsfreqi[0][:]
pdsall[0][5:10] = pdsfreqi[1][:]
pdsall[1][0:5] = pdsfreqc[0][:]
pdsall[1][5:10] = pdsfreqc[1][:]

chi2_stat, p_val, dof, ex = stats.chi2_contingency(pdsall)
print ('chi2 = %f; p = %2.20f' % (chi2_stat, p_val))

chi2 = 49.034000; p = 0.00000016356336427372


In [139]:
a = 5

(sum(cc[pds_m == a]) + sum(cc[pds_f == a]))/(sum(pds_m == a) + sum(pds_f == a))

0.8571428571428571

In [108]:
pdsfreqi

array([[1.041e+03, 7.960e+02, 1.377e+03, 8.500e+01, 5.000e+00],
       [2.683e+03, 8.950e+02, 1.790e+02, 8.000e+00, 1.000e+00]])

In [216]:
from scipy.stats.stats import pearsonr

svol_r, svol_p = pearsonr(all_info.sri24_suptent_supratentorium_volume, all_info.pred)
svolsex_r, svolsex_p = pearsonr(all_info.sri24_suptent_supratentorium_volume, all_info.dx)
age_r, age_p = pearsonr(pd.to_numeric(all_info.interview_age), all_info.pred)

pds = np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_male_category)) + np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_female_category))
pds_r, pds_p = pearsonr(pds, all_info.pred)
pdssex_r, pdssex_p = pearsonr(pds, all_info.dx)

pds = pd.to_numeric(all_info.pds_p_ss_male_category)
idx = ~np.isnan(pds)
pdsm_r, pdsm_p = pearsonr(pds[idx], all_info[idx].pred)

pds = pd.to_numeric(all_info.pds_p_ss_female_category)
idx = ~np.isnan(pds)
pdsf_r, pdsf_p = pearsonr(pds[idx], all_info[idx].pred)

print ('SVol\t %f\t %f \nSVol-Sx\t %f\t %f \nAge\t %f\t %f \nPDS\t %f\t %f' % (svol_r, svol_p, svolsex_r, svolsex_p, age_r, age_p, pds_r, pds_p))
print ('PDS_M\t %f\t %f \nPDS_F\t %f\t %f \nPDS-SEX\t %f\t %f' % (pdsm_r, pdsm_p, pdsf_r, pdsf_p, pdssex_r, pdssex_p))

SVol	 -0.034661	 0.003095 
SVol-Sx	 -0.019381	 0.098179 
Age	 -0.013057	 0.265243 
PDS	 0.373848	 0.000000
PDS_M	 0.053641	 0.001093 
PDS_F	 0.056189	 0.001094 
PDS-SEX	 0.440283	 0.000000


In [40]:
from scipy import stats

allage = pd.to_numeric(all_info['interview_age'])
male = allage[all_info['sex'] == 'M']
female = allage[all_info['sex'] == 'F']
print (np.mean(female/12), np.std(female/12))
print (np.mean(male/12), np.std(male/12))

[sss, p] = stats.ttest_ind(female, male)

print ('%2.20f' % p)


9.920902866923413 0.6193579092708873
9.949694045657766 0.6231226645579144
0.03677028487432058845


In [56]:
pds = (pd.to_numeric(all_info.pds_p_ss_male_category)) + np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_female_category))

# pm = np.nan_to_num(pds[all_info['sex'] == 'M'])
# pf = np.nan_to_num(pds[all_info['sex'] == 'F'])
pf = (pds[all_info['sex'] == 'F'])
pm = (pds[all_info['sex'] == 'M'])

pfnan = np.isnan(pf)
pmnan = np.isnan(pm)

pf = pf[~pfnan]
pm = pm[~pmnan]

print (np.mean(pf), np.std(pf))
print (np.mean(pm), np.std(pm))

[sss, p] = stats.ttest_ind(pf, pm)

print ('%2.20f' % p)

1.5555555555555556 0.8314794192830981
1.3422590068159688 0.5839428476924595
0.27440624941732694175


In [140]:
pred.shape

(8144,)

In [31]:
female/12

0       10.000000
1       10.333333
2       10.333333
3       10.666667
4        9.500000
5       10.083333
7        9.333333
8       10.000000
10      10.500000
12       9.250000
14      10.000000
16      10.583333
17      10.583333
18       9.083333
20       9.750000
22      10.000000
23       9.000000
24       9.666667
25       9.750000
27      10.416667
30      10.250000
32      10.416667
34      10.833333
35       9.333333
38       9.916667
40      10.250000
41       9.833333
42      10.166667
44       9.083333
45       9.166667
          ...    
8092     9.333333
8093     9.750000
8094    10.583333
8096     9.666667
8098     9.500000
8100     9.250000
8101    10.583333
8103    10.000000
8104    10.166667
8107     9.583333
8108    10.250000
8112     9.000000
8113     9.833333
8116     9.166667
8117    10.833333
8118     9.416667
8122    10.583333
8123     9.000000
8124    10.416667
8126     9.083333
8127     9.416667
8128     9.333333
8129    10.000000
8133    10.333333
8136    10

In [85]:
# For features 
import pandas as pd
from functools import reduce

demog = pd.read_csv('abcd_ssphp01.csv')
allsub_demog = demog[demog.subjectkey.isin(subid) & demog.eventname.isin(['baseline_year_1_arm_1'])]

volscores = pd.read_csv('volume_scores.csv')
svol = volscores[['subjectkey', 'sri24_suptent_supratentorium_volume']]
preds = pd.DataFrame({'subjectkey': subid, 'pred': pred, 'dx': dx, 'features': np.empty(dx.shape[0])}, columns=['subjectkey', 'pred', 'dx', 'features'])
fdf = pd.concat([pd.DataFrame(features), pd.DataFrame({'subjectkey': subid}, columns=['subjectkey'])], axis=1)

dfs = [allsub_demog, svol, preds, fdf]

all_info3 = reduce(lambda left,right: pd.merge(left,right,on='subjectkey'), dfs)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [186]:
import dcor
#a = all_info3.svol
np.array([[0, 0], [0, 0]])
b = np.array([0, 0]).T
print(dcor.independence.distance_covariance_test(allf, all_info3.sri24_suptent_supratentorium_volume))

HypothesisTest(p_value=1.0, statistic=6185355.727051117)


In [67]:
fdf = pd.DataFrame(features)
fdf2 = pd.DataFrame({'subjectkey': subid}, columns=['subjectkey'])
fdf3 = pd.concat([fdf, fdf2], axis=1)
#variables = fdf3[0].keys()
#df = pd.DataFrame([[getattr(i,j) for j in variables] for i in fdf3], columns = variables)

In [73]:
fdf = pd.concat([pd.DataFrame(features), pd.DataFrame({'subjectkey': subid}, columns=['subjectkey'])], axis=1)

In [125]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
import copy


def distcorr(Xval, Yval, pval=True, nruns=500):
    """ Compute the distance correlation function, returning the p-value.
    Based on Satra/distcorr.py (gist aa3d19a12b74e9ab7941)
    >>> a = [1,2,3,4,5]
    >>> b = np.array([1,2,9,4,4])
    >>> distcorr(a, b)
    (0.76267624241686671, 0.404)
    """
    X = np.atleast_1d(Xval)
    Y = np.atleast_1d(Yval)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()

    dcov2_xy = (A * B).sum() / float(n * n)
    dcov2_xx = (A * A).sum() / float(n * n)
    dcov2_yy = (B * B).sum() / float(n * n)
    dcorr = np.sqrt(dcov2_xy) / np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))

    if pval:
        greater = 0
        for i in range(nruns):
            Y_r = copy.copy(Yval)
            np.random.shuffle(Y_r)
            if distcorr(Xval, Y_r, pval=False) >= dcorr:
                greater += 1
        return (dcorr, greater / float(nruns))
    else:
        return dcorr

In [127]:
a = np.array([[1, 7, 6], [12, -13, 9]])
b = np.array([0.1, 5])
print(distcorr(a, b))

(1.0, 1.0)


In [ ]:
allf = np.array(all_info3.loc[:, 'features':])

svol_dr, svol_dp = distcorr(allf, all_info3.sri24_suptent_supratentorium_volume, True, 10)
print ('SVOL\t %f\t %f' % (svol_dr, svol_dp))

#age_dr, age_dp = distcorr(allf, pd.to_numeric(all_info.interview_age))
#print ('Age\t %f\t %f' % (age_dr, age_dp))


#pds = np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_male_category)) + np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_female_category))
#pds_r, pds_p = pearsonr(pds, all_info.pred)
#pdssex_r, pdssex_p = pearsonr(pds, all_info.dx)
#pds = pd.to_numeric(all_info.pds_p_ss_male_category)
#idx = ~np.isnan(pds)
#pdsm_r, pdsm_p = pearsonr(pds[idx], all_info[idx].pred)

#pds = pd.to_numeric(all_info.pds_p_ss_female_category)
#idx = ~np.isnan(pds)
#pdsf_r, pdsf_p = pearsonr(pds[idx], all_info[idx].pred)

#print ('SVol\t %f\t %f \nSVol-Sx\t %f\t %f \nAge\t %f\t %f \nPDS\t %f\t %f' % (svol_r, svol_p, svolsex_r, svolsex_p, age_r, age_p, pds_r, pds_p))
#print ('PDS_M\t %f\t %f \nPDS_F\t %f\t %f \nPDS-SEX\t %f\t %f' % (pdsm_r, pdsm_p, pdsf_r, pdsf_p, pdssex_r, pdssex_p))

In [193]:
pds = np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_male_category)) + np.nan_to_num(pd.to_numeric(all_info.pds_p_ss_female_category))
pds_r, pds_p = pearsonr(pds, all_info.pred)
pdsage_r, pdsage_p = pearsonr(pds, pd.to_numeric(all_info.interview_age))
print(pdsage_r, pdsage_p)

0.16241781634632269 3.1370269838170834e-44


In [210]:
min(pd.to_numeric(all_info[all_info.dx==1].interview_age))

108

In [223]:
pds = pd.to_numeric(all_info.pds_p_ss_male_category)
idx = ~np.isnan(pds)
pdsm_r, pdsm_p = pearsonr(pds[idx], all_info[idx].pred)

pds = pd.to_numeric(all_info.pds_p_ss_female_category)
idx = ~np.isnan(pds)
pdsf_r, pdsf_p = pearsonr(pds[idx], all_info[idx].pred)
print (pdsf_p, pdsm_p)


0.001094015775017081 0.0010929209295735941


In [224]:
all_info3.head()

,collection_id,abcd_ssphp01_id,dataset_id,subjectkey,src_subject_id,interview_age,interview_date,sex,eventname,sds_p_ss_dims,...,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607
0,2573,17600,19331,NDAR_INV2LXFHK41,NDAR_INV2LXFHK41,120,10/2/2017,M,baseline_year_1_arm_1,13,...,-0.049024,-0.065625,-0.054688,-0.052010,-0.082267,-0.043763,-0.212164,0.628517,-0.041066,-0.034146
1,2573,17708,19331,NDAR_INV2ZAMNE5F,NDAR_INV2ZAMNE5F,124,6/2/2017,M,baseline_year_1_arm_1,9,...,-0.133299,-0.053839,-0.042890,0.355097,-0.053711,-0.042544,1.152500,1.049183,-0.040615,-0.052084
2,2573,21359,19331,NDAR_INVC5G293GX,NDAR_INVC5G293GX,124,5/6/2018,M,baseline_year_1_arm_1,15,...,-0.049024,-0.065625,-0.054688,-0.052010,-0.082267,-0.043763,0.958606,0.484830,-0.041066,-0.034146
3,2573,18434,19331,NDAR_INV5V348XPG,NDAR_INV5V348XPG,128,7/6/2017,M,baseline_year_1_arm_1,9,...,-0.049024,-0.065625,-0.054688,-0.052010,-0.082267,-0.043763,0.062650,0.769322,-0.041066,-0.034146
4,2573,22496,19331,NDAR_INVF76ZR3W2,NDAR_INVF76ZR3W2,121,9/23/2017,M,baseline_year_1_arm_1,12,...,-0.056402,-0.037780,-0.059252,-0.039381,-0.042442,-0.042856,4.446462,-0.045084,1.685373,0.904782
